<a href="https://colab.research.google.com/github/ncourtel1/strava-urban-mobility-pipeline/blob/main/notebook/01_pipeline_strava.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("heelo")

heelo
